Tasks from https://www.kaggle.com/datasets/ksabishek/product-sales-data?resource=download

1) Is there any trend in the sales of all four products during certain months?
2) Out of all four products , which product has seen the highest sales in all the given years?
3) The company has all it's retail centers closed on the 31st of December every year. Mr: Hariharan , the CEO , would love to get an estimate on no: of units of each product that could be sold on 31st of Dec , every year , if all their retail centers were kept open.
4) The CEO is considering an idea to drop the production of any one of the products. He wants you to analyze this data and suggest whether his idea would result in a massive setback for the company.
5) The CEO would also like to predict the sales and revenues for the year 2024. He wants you to give a yearly estimate with the best possible accuracy.
Can you help REC corp with your analytical and data science skills ?

## Task 1. Is there any trend in the sales of all four products during certain months?

Answer: Yes, there is a huge dip in February with spikes in July and January.

In [41]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("./statsfinal.csv")

df['QTotal'] = df["Q-P1"] + df["Q-P2"] + df["Q-P3"] + df["Q-P4"]
df['dt'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
df['month-year'] = df['dt'].dt.strftime('%Y-%m')
df['month'] = df['dt'].dt.month
df['year'] = df['dt'].dt.year



dx = df.groupby('month-year')["QTotal"].sum()
dy = df.groupby('month')["QTotal"].sum()

dy.head()


month
1.0    4275211
2.0    3561548
3.0    3865316
4.0    3844183
5.0    3954981
Name: QTotal, dtype: int64

In [3]:

fig = px.line(dy, title='Total sales trend')
fig.show()

## Task 2. Out of all four products , which product has seen the highest sales in all the given years?
Answer: product 1

In [4]:

p1 = df["Q-P1"].sum()
p2 = df["Q-P2"].sum()
p3 = df["Q-P3"].sum()
p4 = df["Q-P4"].sum()

dz = pd.DataFrame({"Sales": [p1, p2, p3, p4]}, index=["P1", "P2", "P3", "P4"])

dz.head()

fig = px.bar(dz, title='Total sales by product')
fig.show()


3) The company has all it's retail centers closed on the 31st of December every year. Mr: Hariharan , the CEO , would love to get an estimate on no: of units of each product that could be sold on 31st of Dec , every year , if all their retail centers were kept open.

The easiest way to calculate expected sells on Dec31 is to find an average between sells on Dec30 and Jan1 of each year.

As for more complicated solutions, I would look for linear regression.

Answer: see graph.

In [61]:
subset1 = df[df["Date"].str.startswith("30-12-")][["year","QTotal"]]
subset1.reset_index(inplace=True)

subset1.head()


,index,year,QTotal
0,200,2010.0,17071
1,564,2011.0,12544
2,927,2012.0,9921
3,1291,2013.0,16598
4,1655,2014.0,6459


In [62]:
subset2 = df[df["Date"].str.startswith("01-01-")][["year","QTotal"]]
subset2.reset_index(inplace=True)

subset2["year"] = subset2["year"] - 1

subset2.head()

,index,year,QTotal
0,201,2010.0,9960
1,565,2011.0,13502
2,928,2012.0,9395
3,1292,2013.0,11172
4,1656,2014.0,8289


In [66]:

subset3 = subset1.merge(subset2, on="year", validate="1:1")
subset3["ExpectedSells"] = (subset3["QTotal_x"] + subset3["QTotal_y"] ) / 2

subset3.head()


,index_x,year,QTotal_x,index_y,QTotal_y,ExpectedSells
0,200,2010.0,17071,201,9960,13515.5
1,564,2011.0,12544,565,13502,13023.0
2,927,2012.0,9921,928,9395,9658.0
3,1291,2013.0,16598,1292,11172,13885.0
4,1655,2014.0,6459,1656,8289,7374.0


In [68]:
fig = px.bar(subset3, title='Expected sales on Dec31', x="year", y="ExpectedSells")
fig.show()